**Requirements**

In [ ]:
#!pip install torch==1.4.0
#!pip install torchvision==0.5.0
#!pip install tensorboard==2.6
#!pip install tensorflow-estimator~=2.7
#!pip install tensorflow-gpu==1.15.0
#!pip install scipy==1.5.2
#!pip install Pillow==6.2.2
#!pip install numpy==1.17.0
#!pip install scikit-learn==0.22.1

**Mount Drive**

In [ ]:
from google.colab import drive
drive.mount ('/content/drive')


Mounted at /content/drive


**Setting up the working paths**

In [ ]:
WDIR = '/content/drive/MyDrive/Project/'
#WDIR = '/content/drive/MyDrive/DAAI21/'

**Git Cloning - ROS**

In [ ]:
import os
import shutil

PathROS = WDIR+'ROS_AMLProject'
# If directory doesn't exist it will be created with the following cloning 
if not os.path.isdir(PathROS):
    !git clone https://github.com/silvia1993/ROS_AMLProject.git
    shutil.move('ROS_AMLProject', WDIR)


**Git Cloning - Center Loss**

In [ ]:
PathCL = WDIR+'pytorch-center-loss'
PathCLfile = PathCL+'/center_loss.py'
PathROSCLfile = PathROS+'/center_loss.py'
if not os.path.isdir(PathCL):
    !git clone https://github.com/KaiyangZhou/pytorch-center-loss.git
    shutil.move('pytorch-center-loss',WDIR)
    shutil.copyfile(PathCLfile,PathROSCLfile)

**Load (unzip) dataset if necessary**

In [ ]:
from zipfile import ZipFile as ZIP

PathData = PathROS+'/data'
PathOH = PathROS+'/data/OfficeHome/'
PathZip = WDIR+'OfficeHome.zip'
if not os.path.isdir(PathOH):
    !unzip -u {PathZip} -d {PathData+'/'}
    !mv {PathData+'/OfficeHomeDataset_10072016'} {PathData+'/OfficeHome'}

Output streaming troncato alle ultime 5000 righe.
  inflating: /content/drive/MyDrive/Project/ROS_AMLProject/data/OfficeHomeDataset_10072016/Product/Speaker/00050.jpg  
  inflating: /content/drive/MyDrive/Project/ROS_AMLProject/data/OfficeHomeDataset_10072016/Product/Speaker/00051.jpg  
  inflating: /content/drive/MyDrive/Project/ROS_AMLProject/data/OfficeHomeDataset_10072016/Product/Speaker/00052.jpg  
  inflating: /content/drive/MyDrive/Project/ROS_AMLProject/data/OfficeHomeDataset_10072016/Product/Speaker/00053.jpg  
  inflating: /content/drive/MyDrive/Project/ROS_AMLProject/data/OfficeHomeDataset_10072016/Product/Speaker/00054.jpg  
  inflating: /content/drive/MyDrive/Project/ROS_AMLProject/data/OfficeHomeDataset_10072016/Product/Speaker/00055.jpg  
  inflating: /content/drive/MyDrive/Project/ROS_AMLProject/data/OfficeHomeDataset_10072016/Product/Speaker/00056.jpg  
  inflating: /content/drive/MyDrive/Project/ROS_AMLProject/data/OfficeHomeDataset_10072016/Product/Speaker/00057.jpg 

**New text lists**

In [ ]:
PathNTL = PathROS+'/new_txt_list'
if not os.path.isdir(PathNTL):
    os.mkdir(PathNTL)

**Running the code**

In [ ]:
!pwd
%cd {PathROS}
File = PathROS+'/main.py'

!python {File} --path_dataset {PathData+'/'} --weight_RotTask_step1 1.5 --weight_RotTask_step2 1.5 --threshold -0.4 --weight_Center_Loss 0 --source 'Clipart' --target 'RealWorld' --batch_size 32 --epochs_step1 12 --epochs_step2 12 --learning_rate 0.001


/content
/content/drive/.shortcut-targets-by-id/1-to_oHIF9xIqICIjh9nHsb4JC2muHJ7L/Project/ROS_AMLProject
Args obtained
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100% 44.7M/44.7M [00:00<00:00, 64.3MB/s]
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Source:  Clipart  Target:  RealWorld
Dataset size: source 3064, target 4357
Batch size:  32
Openness: 0.307692
Rotation task 1 at step 1: 1.500000
Rotation task 2 at step 1: 1.500000
Known/Unknown separation threshold: -0.400000
Learning rate: 0.00100

In [ ]:
import torch
import numpy as np
from sklearn.metrics import roc_auc_score


a = np.array([1, 0, 1, 0, 1, 0])
b = np.array([0.5, 1, 0.1, 0.1, 0.1, 0.1])
c = roc_auc_score(a,b)
c
int(c)

from zipfile import ZipFile as ZIP
ZIP(PathZip).extractall()

PathROS = WDIR+'ROS_AMLProject/'
PathOH = PathROS+'data/OfficeHome'
PathZip = WDIR+'OfficeHome.zip'






In [ ]:
            features_rot = feature_extractor(data_rot)
            rot_pred = rot_cls(torch.cat((features,features_rot),1))
            # Prediction
            Zi = torch.nn.Softmax(dim=-1)(rot_pred)
            normality_scorei = torch.max(Zi)

NameError: ignored

In [ ]:

import torch
import numpy as np
from sklearn.metrics import roc_auc_score
import random
import tensorflow as tf
import os


#### Implement the evaluation on the target for the known/unknown separation

def evaluation(args,feature_extractor,rot_cls,target_loader_eval,device):

    feature_extractor.eval()
    rot_cls.eval()

    k = len(target_loader_eval.dataset) # Number of images
    normality_scorei = torch.tensor([0],device=device)

    EntropyScore = torch.zeros((k,4), device=device)
    RotScore = torch.zeros((k,4), device=device)

    ground_truth = torch.zeros((k,), device=device)
    Pred_UK = torch.zeros((k,), device=device)
    normality_score = torch.zeros((k,), device=device)

    # Call the number of known classes constant, take its log and make it a tensor
    logCS = (torch.as_tensor(np.log(args.n_classes_known),dtype=torch.float32)).to(device)

    with torch.no_grad():
        for it, (data,class_l,data_rot,rot_l) in enumerate(target_loader_eval):
            #print("----------------------- Eval for loop start print -----------------------")
            data, class_l, data_rot, rot_l = data.to(device), class_l.to(device), data_rot.to(device), rot_l.to(device)

            features = feature_extractor(data)
            # ------------------------------------------------------------------
            # Explore all possible rotations
            # ------------------------------------------------------------------

            Zi_r = torch.zeros((1,4), device=device)
            entropy_i = torch.zeros((1,4), device=device)
            Zi_r = torch.as_tensor(Zi_r, dtype=torch.float32, device=device)
            entropy_i = torch.as_tensor(entropy_i, dtype=torch.float32, device=device)
            for rot_it in range(4):
                # Compute rotation rot_it
                data_rot = torch.rot90(data,k=rot_it,dims=[2,3])
                rot_l = rot_it
                #print(data.size()) = [1, 3, 222, 222], print(data_rot.size()) = [1, 3, 222, 222]

                # Pass rotation
                features_rot = feature_extractor(data_rot)
                rot_pred = rot_cls(torch.cat((features,features_rot),1))
                # Prediction
                Zi = torch.nn.Softmax(dim=-1)(rot_pred)           # p_rot = Zi

                #------------------- Computing the Scores -------------------
                entropy = (Zi)*torch.log((Zi) + 1e-21)/logCS      # entropy = H(Zi)
                #print(Zi.size()) = [1 4], print(entropy.size()) = [1 4]
   
                Zi_r += Zi
                entropy_i += entropy

            # ------------------------------------------------------------------
            # Managing data from all rotations
            # ------------------------------------------------------------------

            #print(Zi_r.size()) = [1 4], print(entropy_i.size()) = [1 4]
            RotScore[it] = Zi_r
            EntropyScore[it] = (entropy_i/4)

            if (class_l >= args.n_classes_known):
                ground_truth[it] = 0
            else:
                ground_truth[it] = 1


    #EntropyScore = (EntropyScore - torch.min(EntropyScore))/(torch.max(EntropyScore) - torch.min(EntropyScore))
    #EntropyScore = 1 - EntropyScore
    RS_std,RS_mean = torch.std_mean(RotScore)
    RotScore = (RotScore - RS_mean)/RS_std
    EntropyScore = 1 - EntropyScore
    ES_std,ES_mean = torch.std_mean(EntropyScore)
    EntropyScore = (EntropyScore - ES_mean)/ES_std
    print('Entropy Score: ', str(EntropyScore))
    print('Rotation Score: ', str(RotScore))
    RotScore = np.mean(np.array(RotScore.cpu()),axis=0)
    EntropyScore = np.mean(np.array(EntropyScore.cpu()),axis=0)
    normality_score = np.maximum(RotScore,EntropyScore)
    #normality_score = np.mean(normality_score,dim=1)

    # --------------------------------------------------------------------------
    # Separate known from unknown and calculate the AUROC score ----------------
    # --------------------------------------------------------------------------
    #normality_score = (normality_score - torch.min(normality_score))/(torch.max(normality_score) - torch.min(normality_score))
    NS_std = np.std(normality_score)
    NS_mean = np.mean(normality_score)
    normality_score = (normality_score - NS_mean)/NS_std
    normality_score = normality_score[np.logical_not(np.isnan(normality_score))]
    print('Normality Score: ', str(normality_score))
    #ground_truth = (normality_score>args.threshold)
    print('Mean value of the normality score',np.mean(normality_score))
    print(normality_score)

    for it1 in range(len(int(normality_score))):
        if (normality_score[it1] > args.threshold):
            Pred_UK[it1] = 1
        else:
            Pred_UK[it1] = 0
    
    print(ground_truth, Pred_UK)

    normality_score_cpu = normality_score.cpu()
    ground_truth_cpu = ground_truth.cpu()
    normality_score_cpu = np.array(normality_score_cpu)
    ground_truth_cpu = np.array(ground_truth_cpu)

    auroc = roc_auc_score(ground_truth_cpu,normality_score_cpu)
    print('   AUROC %.4f' % auroc)

    #if (auroc < 0.5):
    #    stop

    # --------------------------------------------------------------------------
    # Update the known and unknown samples at the source and target files ------
    # --------------------------------------------------------------------------
    number_of_known_samples = (torch.sum(Pred_UK)).int()
    number_of_unknown_samples = (k - torch.sum(Pred_UK)).int()

    print('   The number of target samples selected as known is: ', number_of_known_samples,' of ',torch.sum((ground_truth),0).int())
    print('   The number of target samples selected as unknown is: ', number_of_unknown_samples, ' of ', (k - (torch.sum((ground_truth),0))).int())

    images_labels = target_loader_eval.dataset.labels 
    images_paths = target_loader_eval.dataset.names

    known_images_paths, known_images_labels, unknown_images_paths = [], [], []

    # Known/unknown paths and labels for evaluated images
    for it2 in range(k):
        if (Pred_UK[it2]==1):
            known_images_labels = known_images_labels+[images_labels[it2]]
            known_images_paths = known_images_paths+[images_paths[it2]]
        else:
            unknown_images_paths = unknown_images_paths+[images_paths[it2]]

    # Create new txt files
    rand = random.randint(0,100000)
    rand = 19
    print('   Generated random number is :', rand)
    if not os.path.isdir('/content/drive/MyDrive/DAAI21/ROS_AMLProject/new_txt_list/'):
        os.mkdir('/content/drive/MyDrive/DAAI21/ROS_AMLProject/new_txt_list/')

    # This txt files will have the names of the source images and the names of the target images selected as unknown
    # According to bibliography: [Ds_known + Dt_unknown], Currently implemented: [Ds_known + Dt_unknown]
    target_unknown = open('new_txt_list/'+args.source+'_unknown_'+str(rand)+'.txt','w')
    for it3 in range (number_of_unknown_samples):
        target_unknown.write(unknown_images_paths[it3]+' '+str(args.n_classes_known)+'\n')
        # Label will always be 45, since that indicates the unknown class label

    files = ['txt_list/'+args.source+'_known'+'.txt','new_txt_list/'+args.source+'_unknown_'+str(rand)+'.txt']
    # Open file3 in write mode
    with open('new_txt_list/'+args.source+'_known_'+str(rand)+'.txt', 'w') as outfile:
        for names in files:
            # Open each file in read mode
            with open(names) as infile:
                # read the data from Ds and Dt_unknown and write [Ds U Dt_unknown]
                outfile.write(infile.read())

    # This txt files will have the names of the target images selected as known
    # According to bibliography:[Dt_known], Currently implemented:[Dt_known]
    target_known = open('new_txt_list/'+args.target+'_known__'+str(rand)+'.txt','w')
    for it4 in range (number_of_known_samples):
        target_known.write(known_images_paths[it4]+' '+str(known_images_labels[it4])+'\n')

    return rand





**Shared**

In [ ]:

import torch
import numpy as np
from sklearn.metrics import roc_auc_score
import random
import tensorflow as tf
import os


#### Implement the evaluation on the target for the known/unknown separation

def evaluation(args,feature_extractor,rot_cls,target_loader_eval,device):

    feature_extractor.eval()
    rot_cls.eval()

    k = len(target_loader_eval.dataset) # Number of images
    normality_scorei = torch.tensor([0],device=device)

    EntropyScore = torch.zeros((k,4), device=device)
    RotScore = torch.zeros((k,4), device=device)

    ground_truth = torch.zeros((k,), device=device)
    Pred_UK = torch.zeros((k,), device=device)
    normality_score = torch.zeros((k,), device=device)

    # Call the number of known classes constant, take its log and make it a tensor
    logCS = (torch.as_tensor(np.log(args.n_classes_known),dtype=torch.float32)).to(device)

    with torch.no_grad():
        for it, (data,class_l,data_rot,rot_l) in enumerate(target_loader_eval):
            #print("----------------------- Eval for loop start print -----------------------")
            data, class_l, data_rot, rot_l = data.to(device), class_l.to(device), data_rot.to(device), rot_l.to(device)

            features = feature_extractor(data)
            # ------------------------------------------------------------------
            # Explore all possible rotations
            # ------------------------------------------------------------------

            Zi_r = torch.zeros((1,4), device=device)
            entropy_i = torch.zeros((1,4), device=device)
            Zi_r = torch.as_tensor(Zi_r, dtype=torch.float32, device=device)
            entropy_i = torch.as_tensor(entropy_i, dtype=torch.float32, device=device)
            for rot_it in range(4):
                # Compute rotation rot_it
                data_rot = torch.rot90(data,k=rot_it,dims=[2,3])
                rot_l = rot_it
                #print(data.size()) = [1, 3, 222, 222], print(data_rot.size()) = [1, 3, 222, 222]

                # Pass rotation
                features_rot = feature_extractor(data_rot)
                rot_pred = rot_cls(torch.cat((features,features_rot),1))
                # Prediction
                Zi = torch.nn.Softmax(dim=-1)(rot_pred)           # p_rot = Zi

                #------------------- Computing the Scores -------------------
                entropy = (Zi)*torch.log((Zi) + 1e-21)/logCS      # entropy = H(Zi)
                #print(Zi.size()) = [1 4], print(entropy.size()) = [1 4]
   
                Zi_r += Zi
                entropy_i += entropy

            # ------------------------------------------------------------------
            # Managing data from all rotations
            # ------------------------------------------------------------------

            #print(Zi_r.size()) = [1 4], print(entropy_i.size()) = [1 4]
            RotScore[it] = Zi_r
            EntropyScore[it] = (entropy_i/4)

            #normality_scorei = torch.maximum(RotScore,EntropyScore)
            #normality_score[it] = normality_scorei
            #normality_score[it] = torch.max(RotScore)

            if (class_l >= args.n_classes_known):
                ground_truth[it] = 0
            else:
                ground_truth[it] = 1


    #EntropyScore = (EntropyScore - torch.min(EntropyScore))/(torch.max(EntropyScore) - torch.min(EntropyScore))
    #EntropyScore = 1 - EntropyScore
    RS_std,RS_mean = torch.std_mean(RotScore)
    RotScore = (RotScore - RS_mean)/RS_std
    EntropyScore = 1 - EntropyScore
    ES_std,ES_mean = torch.std_mean(EntropyScore)
    EntropyScore = (EntropyScore - ES_mean)/ES_std
    print('Entropy Score: ', str(EntropyScore))
    print('Rotation Score: ', str(RotScore))
    normality_score = torch.maximum(RotScore,EntropyScore)
    normality_score = torch.mean(normality_score,dim=1)
    print('Size of the normality score, it should be [k, 1]', normality_score.size())


    # --------------------------------------------------------------------------
    # Separate known from unknown and calculate the AUROC score ----------------
    # --------------------------------------------------------------------------
    #normality_score = (normality_score - torch.min(normality_score))/(torch.max(normality_score) - torch.min(normality_score))
    NS_std,NS_mean = torch.std_mean(normality_score)
    normality_score = (normality_score - NS_mean)/NS_std
    print('Normality Score: ', str(normality_score))
    #ground_truth = (normality_score>args.threshold)
    print('Mean value of the normality score',torch.mean(normality_score))
    print(normality_score)

    for it1 in range(len(normality_score)):
        if (normality_score[it1] > args.threshold):
            Pred_UK[it1] = 1
        else:
            Pred_UK[it1] = 0
    
    print(ground_truth, Pred_UK)

    normality_score_cpu = normality_score.cpu()
    ground_truth_cpu = ground_truth.cpu()
    normality_score_cpu = np.array(normality_score_cpu)
    ground_truth_cpu = np.array(ground_truth_cpu)

    auroc = roc_auc_score(ground_truth_cpu,normality_score_cpu)
    print('   AUROC %.4f' % auroc)

    #if (auroc < 0.5):
    #    stop

    # --------------------------------------------------------------------------
    # Update the known and unknown samples at the source and target files ------
    # --------------------------------------------------------------------------
    number_of_known_samples = (torch.sum(Pred_UK)).int()
    number_of_unknown_samples = (k - torch.sum(Pred_UK)).int()

    print('   The number of target samples selected as known is: ', number_of_known_samples,' of ',torch.sum((ground_truth),0).int())
    print('   The number of target samples selected as unknown is: ', number_of_unknown_samples, ' of ', (k - (torch.sum((ground_truth),0))).int())

    images_labels = target_loader_eval.dataset.labels 
    images_paths = target_loader_eval.dataset.names

    known_images_paths, known_images_labels, unknown_images_paths = [], [], []

    # Known/unknown paths and labels for evaluated images
    for it2 in range(k):
        if (Pred_UK[it2]==1):
            known_images_labels = known_images_labels+[images_labels[it2]]
            known_images_paths = known_images_paths+[images_paths[it2]]
        else:
            unknown_images_paths = unknown_images_paths+[images_paths[it2]]

    # Create new txt files
    rand = random.randint(0,100000)
    rand = 1
    print('   Generated random number is :', rand)
    if not os.path.isdir('/content/drive/MyDrive/DAAI21/ROS_AMLProject/new_txt_list/'):
        os.mkdir('/content/drive/MyDrive/DAAI21/ROS_AMLProject/new_txt_list/')

    # This txt files will have the names of the source images and the names of the target images selected as unknown
    # According to bibliography: [Ds_known + Dt_unknown], Currently implemented: [Ds_known + Dt_unknown]
    target_unknown = open('new_txt_list/'+args.source+'_unknown_'+str(rand)+'.txt','w')
    for it3 in range (number_of_unknown_samples):
        target_unknown.write(unknown_images_paths[it3]+' '+str(args.n_classes_known)+'\n')
        # Label will always be 45, since that indicates the unknown class label

    files = ['txt_list/'+args.source+'_known'+'.txt','new_txt_list/'+args.source+'_unknown_'+str(rand)+'.txt']
    # Open file3 in write mode
    with open('new_txt_list/'+args.source+'_known_'+str(rand)+'.txt', 'w') as outfile:
        for names in files:
            # Open each file in read mode
            with open(names) as infile:
                # read the data from Ds and Dt_unknown and write [Ds U Dt_unknown]
                outfile.write(infile.read())

    # This txt files will have the names of the target images selected as known
    # According to bibliography:[Dt_known], Currently implemented:[Dt_known]
    target_known = open('new_txt_list/'+args.target+'_known__'+str(rand)+'.txt','w')
    for it4 in range (number_of_known_samples):
        target_known.write(known_images_paths[it4]+' '+str(known_images_labels[it4])+'\n')

    return rand



